<img src = "https://8weeksqlchallenge.com/images/case-study-designs/3.png">

## Introduction

Subscription based businesses are super popular and Danny realised that there was a large gap in the market - he wanted to create a new streaming service that only had food related content - something like Netflix but with only cooking shows!

## Problem Statement

Danny finds a few smart friends to launch his new startup Foodie-Fi in 2020 and started selling monthly and annual subscriptions, giving their customers unlimited on-demand access to exclusive food videos from around the world!

Danny created Foodie-Fi with a data driven mindset and wanted to ensure all future investment decisions and new features were decided using data. This case study focuses on using subscription style digital data to answer important business questions.


## Entity Relationship Diagram

<img src = "https://8weeksqlchallenge.com/images/case-study-3-erd.png">

In [1]:
import sqlite3
import pandas as pd

In [2]:
database = '../data/FoodieFi.sqlite'

In [3]:
connection = sqlite3.connect(database)

# Case Study Questions

---

## B. Data Analysis Questions


---

### 1. How many customers has Foodie-Fi ever had?

In [7]:
pd.read_sql("""

            SELECT 
                COUNT(DISTINCT(customer_id)) AS num_customers 
            FROM subscriptions; 
            
            """,connection)

,num_customers
0,1000


### 2. What is the monthly distribution of trial plan start_date values for our dataset - use the start of the month as the group by value

In [8]:
pd.read_sql("""

            SELECT strftime('%m', start_date) as Month,
            COUNT(customer_id) AS trial_subscriptions
            FROM subscriptions s
            INNER JOIN plans p
              ON s.plan_id = p.plan_id
            WHERE s.plan_id = 0
            GROUP BY Month
            ORDER BY Month;

        """,connection)

,Month,trial_subscriptions
0,01,88
1,02,68
2,03,94
3,04,81
4,05,88
5,06,79
6,07,89
7,08,88
8,09,87
9,10,79


### 3. What plan start_date values occur after the year 2020 for our dataset? Show the breakdown by count of events for each plan_name

In [9]:
pd.read_sql("""

            SELECT 
              p.plan_id,
              p.plan_name,
              COUNT(*) AS events
            FROM subscriptions s
            JOIN plans p
              ON s.plan_id = p.plan_id
            WHERE s.start_date >= '2021-01-01'
            GROUP BY p.plan_id, p.plan_name
            ORDER BY p.plan_id;
            
            """,connection)

,plan_id,plan_name,events
0,1,basic monthly,8
1,2,pro monthly,60
2,3,pro annual,63
3,4,churn,71


### 4. What is the customer count and percentage of customers who have churned rounded to 1 decimal place?

In [12]:
pd.read_sql("""

            SELECT 
              COUNT(*) AS churn_count,
              ROUND(100 * COUNT(*) / (
                SELECT COUNT(DISTINCT customer_id) 
                FROM subscriptions),1) AS churn_percentage
            FROM subscriptions s
            JOIN plans p
              ON s.plan_id = p.plan_id
            WHERE s.plan_id = 4;

            """,connection)

,churn_count,churn_percentage
0,307,30.0


### 5. How many customers have churned straight after their initial free trial - what percentage is this rounded to the nearest whole number?

In [13]:
pd.read_sql("""

            WITH churn_ranking AS 
            (
            SELECT 
              s.customer_id, 
              s.plan_id, 
              p.plan_name,
              ROW_NUMBER() OVER (PARTITION BY s.customer_id ORDER BY s.plan_id) AS plan_rank
            FROM subscriptions s
            JOIN plans p
              ON s.plan_id = p.plan_id
            )

            SELECT 
              COUNT(*) AS churn_count,
              ROUND(100 * COUNT(*) / (
                SELECT COUNT(DISTINCT customer_id) 
                FROM subscriptions),0) AS churn_percentage
            FROM churn_ranking
            WHERE plan_id = 4
              AND plan_rank = 2;
            
            """,connection)

,churn_count,churn_percentage
0,92,9.0


### 6. What is the number and percentage of customer plans after their initial free trial?

In [14]:
pd.read_sql("""
            
            WITH next_plan AS (
            SELECT 
              customer_id, 
              plan_id, 
              LEAD(plan_id, 1) OVER(PARTITION BY customer_id ORDER BY plan_id) as next_plan
            FROM subscriptions)

            SELECT 
              next_plan, 
              COUNT(*) AS conversions,
              ROUND(100 * COUNT(*) / (
                SELECT COUNT(DISTINCT customer_id) 
                FROM subscriptions),1) AS conversion_percentage
            FROM next_plan
            WHERE next_plan IS NOT NULL 
              AND plan_id = 0
            GROUP BY next_plan
            ORDER BY next_plan;

            """,connection)

,next_plan,conversions,conversion_percentage
0,1,546,54.0
1,2,325,32.0
2,3,37,3.0
3,4,92,9.0


### 7.  What is the customer count and percentage breakdown of all 5 plan_name values at 2020-12-31?

In [15]:
pd.read_sql("""

            WITH 

            next_plan AS(
                SELECT 
                  customer_id, 
                  plan_id, 
                  start_date,
                  LEAD(start_date, 1) OVER(PARTITION BY customer_id ORDER BY start_date) as next_date
                FROM subscriptions
                WHERE start_date <= '2020-12-31'),

            -- To find breakdown of customers with existing plans on or after 31 Dec 2020

            customer_breakdown AS (
              SELECT plan_id, COUNT(DISTINCT customer_id) AS customers
                FROM next_plan
                WHERE (next_date IS NOT NULL AND (start_date < '2020-12-31' AND next_date > '2020-12-31'))
                  OR (next_date IS NULL AND start_date < '2020-12-31')
                GROUP BY plan_id)

            SELECT plan_id, customers, 
              ROUND(100 * customers / (
                SELECT COUNT(DISTINCT customer_id) 
                FROM subscriptions),1) AS percentage
            FROM customer_breakdown
            GROUP BY plan_id, customers
            ORDER BY plan_id


            """,connection)

,plan_id,customers,percentage
0,0,19,1.0
1,1,224,22.0
2,2,326,32.0
3,3,195,19.0
4,4,235,23.0


### 8. How many customers have upgraded to an annual in 2020?

In [16]:
pd.read_sql("""

            SELECT 
              COUNT(DISTINCT customer_id) AS unique_customer
            FROM subscriptions
            WHERE plan_id = 3
              AND start_date <= '2020-12-31';

            """,connection)

,unique_customer
0,195


### 9. How many days on average does it take for a customer to an annual plan from the day they join Foodie-Fi?

In [17]:
pd.read_sql("""

        WITH 
          trial_plan AS 
          (SELECT 
              customer_id, 
              start_date AS trial_date
          FROM subscriptions
          WHERE plan_id = 0
          ),

          annual_plan AS
          (SELECT 
              customer_id, 
              start_date AS annual_date
          FROM subscriptions
          WHERE plan_id = 3
          )

        SELECT 
            ROUND(AVG(JULIANDAY(ap.annual_date) - JULIANDAY(tp.trial_date))) AS avg_days_to_upgrade
        FROM trial_plan tp
        INNER JOIN annual_plan ap
            ON tp.customer_id = ap.customer_id
    

            """,connection)

,avg_days_to_upgrade
0,105.0


### 10.  Can you further breakdown this average value into 30 day periods (i.e. 0-30 days, 31-60 days etc)

In [18]:
pd.read_sql("""

            WITH 
              trial_plan AS 
              (SELECT 
                  customer_id, 
                  start_date AS trial_date
              FROM subscriptions
              WHERE plan_id = 0
              ),

              annual_plan AS
              (SELECT 
                  customer_id, 
                  start_date AS annual_date
              FROM subscriptions
              WHERE plan_id = 3
              ), 

              day_period AS
                (SELECT 
                JULIANDAY(ap.annual_date) - JULIANDAY(tp.trial_date) AS diff
                FROM trial_plan tp
                INNER JOIN annual_plan ap
                    ON tp.customer_id = ap.customer_id
                WHERE annual_date IS NOT NULL
                ),

              bins AS
              (SELECT 
                  *, FLOOR(diff/30) AS bins
                FROM day_period
              )

            SELECT ((bins*30)+1) || ' - ' || ((bins+1)*30) || ' days' AS Days,
            COUNT(diff) AS Total
            FROM bins
            GROUP BY bins;


            """,connection)

,Days,Total
0,1.0 - 30.0 days,48
1,31.0 - 60.0 days,25
2,61.0 - 90.0 days,33
3,91.0 - 120.0 days,35
4,121.0 - 150.0 days,43
5,151.0 - 180.0 days,35
6,181.0 - 210.0 days,27
7,211.0 - 240.0 days,4
8,241.0 - 270.0 days,5
9,271.0 - 300.0 days,1


### 11. How many customers downgraded from a pro monthly to a basic monthly plan in 2020?

In [19]:
pd.read_sql("""

        WITH next_plan_cte AS (
        SELECT 
          customer_id, 
          plan_id, 
          start_date,
          LEAD(plan_id, 1) OVER(PARTITION BY customer_id ORDER BY plan_id) as next_plan
        FROM subscriptions)

        SELECT 
          COUNT(*) AS downgraded
        FROM next_plan_cte
        WHERE start_date <= '2020-12-31'
          AND plan_id = 2 
          AND next_plan = 1;

""",connection)

,downgraded
0,0
